In [35]:
import numpy as np
import pandas as pd

# Define Rastrigin Function


In [74]:
def rastrigin(X):
    val = []
    for xi in X:
        fx = 10 * len(np.array(xi)) - sum(np.array(xi) ** 2 - 10 * np.cos(2 * np.pi * np.array(xi)))
        val.append(fx)
    return np.array(val)

In [75]:
input_values = [
    [0, 0],
    [1, 1],
    [-1, -1],
    [2, 2],
    [-2, -2],
    [0.5, 0.5],
    [-0.5, -0.5],
    [3, 3],
    [-3, -3],
    [2.5, -2.5]
]

rastrigin(np.array(input_values))

array([ 40. ,  38. ,  38. ,  32. ,  32. ,  -0.5,  -0.5,  22. ,  22. ,
       -12.5])

In [76]:
input_values_1d = [0, 1, -1, 2, -2, 0.5, -0.5, 3, -3, 2.5]
rastrigin(np.array(input_values_1d))

TypeError: len() of unsized object

In [82]:
for i in input_values_1d:
    print ((np.array(i.tolist())))


AttributeError: 'int' object has no attribute 'tolist'

In [83]:
a = 5
a.toList()

AttributeError: 'int' object has no attribute 'toList'

# Initiate population

In [41]:
def generate_particle(num_variables,population_size,x_min,x_max):
    X = []
    for i in range(population_size):
        Xi = []
        for j in range(num_variables):
            Xi.append(np.random.uniform(x_min,x_max))
        X.append(Xi)
        
    return np.array(X)

In [61]:
np.sort(rastrigin(generate_particle(4,5,-5,5).T))

array([ 73.10713332,  86.98445797, 101.93452027, 102.46343908,
       102.56750544])

# Define Particle Swarm Optimisation

In [7]:
def pso(num_generation,population_size,num_variables):
    
    
    pass


        